In [35]:
import os
import sys 
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow import keras
from tensorflow.keras.layers import Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

import plotly
import plotly.graph_objects as go
import plotly.express as px
import kaleido ##pip install -U kaleido ##to save a plotly fig
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score,cross_val_predict

In [37]:
df = pd.read_csv('longitudinal behavioral data.csv')
df=df.drop(['subject','2weeks_class','1month_class','2months_class','3months_class'],axis=1)
#df.dtypes[df.dtypes==object]
df['Annoying.4'] = df['Annoying.4'].replace('.', 2)
df['Annoying.4'] = df['Annoying.4'].astype('float32')
print(len(df))
#df.columns.get_loc("3months_TFI") -->80

18


In [38]:
## Separate dependent and independent features 
X=df.iloc[:,:80].values
y=df.iloc[:,-20:].values

In [39]:
# with scaled data
n_epochs=200
n_batch_size=6 #batch training
n_timesteps=4
n_features=20
lr=0.001
kfold = KFold(n_splits=5)
error_scores=[]

# Reshape input to be 3D for LSTM[samples, timesteps, features]
X = X.reshape((X.shape[0],n_timesteps,n_features))
print(X.shape,y.shape)
input_shape=(n_timesteps,n_features)
print(input_shape)
#sc_x=StandardScaler()
#sc_y=StandardScaler()
callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

def create_lstm_model(input_shape):#,y_scaler
       
    K.clear_session
    input_shape=input_shape
    model = Sequential([
      LSTM(50, activation='relu',return_sequences=True,input_shape=input_shape),
      Dropout(0.2),
      LSTM(20, activation='relu',input_shape=input_shape),
      Dropout(0.2),
      Dense(1,kernel_initializer='normal',activation = 'linear')                      
    ])             

    model.compile(loss='mae', optimizer=tf.keras.optimizers.Adam(learning_rate=lr))#run_eagerly=True,
    return model
    
def cross_val(df,error_scores):
    #X,y,input_shape,x_scaler,y_scaler=data_prep(df)
    for train_index, test_index in kfold.split(X):
        print("TRAIN:", train_index, "TEST:", test_index)
        x_train, x_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model=create_lstm_model(input_shape)#,y_scaler
        history=model.fit(x_train, y_train,epochs=n_epochs,validation_split=0.2,batch_size=n_batch_size,callbacks=callbacks,verbose =1)#validation_data=(x_test,y_test) #validation_split=0.2
        #val_mse,val_mae=model.evaluate(x_test,y_test) #evaluating using unseen data, bcz validation is not used while model.fit
        val_mse=model.evaluate(x_test,y_test)
        error_scores.append(val_mse)
    return history,error_scores

history,error_scores=cross_val(df,error_scores)

    

(18, 4, 20) (18, 20)
(4, 20)
TRAIN: [ 4  5  6  7  8  9 10 11 12 13 14 15 16 17] TEST: [0 1 2 3]
Epoch 1/200


2022-06-09 06:45:49.326917: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 346ms/step - loss: 12.5313 - val_loss: 24.5110
Epoch 2/200
1/2 [==============>...............] - ETA: 0s - loss: 13.2332

2022-06-09 06:45:50.012113: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 0s 92ms/step - loss: 12.2831 - val_loss: 22.9935
Epoch 3/200
2/2 [==============================] - 0s 91ms/step - loss: 11.6546 - val_loss: 22.2353
Epoch 4/200
2/2 [==============================] - 0s 84ms/step - loss: 11.3723 - val_loss: 21.7635
Epoch 5/200
2/2 [==============================] - 0s 84ms/step - loss: 11.0323 - val_loss: 21.4311
Epoch 6/200
2/2 [==============================] - 0s 82ms/step - loss: 11.1091 - val_loss: 21.2719
Epoch 7/200
2/2 [==============================] - 0s 76ms/step - loss: 10.8947 - val_loss: 21.1404
Epoch 8/200
2/2 [==============================] - 0s 77ms/step - loss: 11.0966 - val_loss: 21.0254
Epoch 9/200
2/2 [==============================] - 0s 80ms/step - loss: 10.6887 - val_loss: 20.8136
Epoch 10/200
2/2 [==============================] - 0s 75ms/step - loss: 10.6651 - val_loss: 20.8539
Epoch 11/200
2/2 [==============================] - 0s 80ms/step - loss: 10.5576 - val_loss: 20.9216
Epoch 12/2

...

2/2 [==============================] - 0s 70ms/step - loss: 11.9163 - val_loss: 22.0244
Epoch 3/200
2/2 [==============================] - 0s 64ms/step - loss: 10.8498 - val_loss: 21.5399
Epoch 4/200
2/2 [==============================] - 0s 60ms/step - loss: 10.4491 - val_loss: 21.3819
Epoch 5/200
2/2 [==============================] - 0s 59ms/step - loss: 10.2841 - val_loss: 21.2445
Epoch 6/200
2/2 [==============================] - 0s 58ms/step - loss: 10.0841 - val_loss: 21.1217
Epoch 7/200
2/2 [==============================] - 0s 64ms/step - loss: 10.0678 - val_loss: 20.8619
Epoch 8/200
2/2 [==============================] - 0s 55ms/step - loss: 9.9826 - val_loss: 20.6281
Epoch 9/200
2/2 [==============================] - 0s 58ms/step - loss: 10.0062 - val_loss: 20.6408
Epoch 10/200
2/2 [==============================] - 0s 55ms/step - loss: 10.0335 - val_loss: 20.6539
Epoch 11/200
2/2 [==============================] - 0s 56ms/step - loss: 10.0284 - val_loss: 20.6809
Epoch 12/20

2022-06-09 06:46:08.663194: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 1s 526ms/step - loss: 12.4510 - val_loss: 12.7411
Epoch 2/200
1/2 [==============>...............] - ETA: 0s - loss: 10.1723

2022-06-09 06:46:09.423369: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2/2 [==============================] - 0s 67ms/step - loss: 11.8956 - val_loss: 12.1119
Epoch 3/200
2/2 [==============================] - 0s 67ms/step - loss: 11.0868 - val_loss: 11.3323
Epoch 4/200
2/2 [==============================] - 0s 56ms/step - loss: 10.6312 - val_loss: 11.2122
Epoch 5/200
2/2 [==============================] - 0s 55ms/step - loss: 10.2222 - val_loss: 11.1189
Epoch 6/200
2/2 [==============================] - 0s 57ms/step - loss: 10.4460 - val_loss: 11.0340
Epoch 7/200
2/2 [==============================] - 0s 56ms/step - loss: 10.1387 - val_loss: 10.9646
Epoch 8/200
2/2 [==============================] - 0s 54ms/step - loss: 10.2234 - val_loss: 10.9871
Epoch 9/200
2/2 [==============================] - 0s 58ms/step - loss: 10.0664 - val_loss: 11.0421
Epoch 10/200
2/2 [==============================] - 0s 57ms/step - loss: 10.0416 - val_loss: 11.0546
Epoch 11/200
2/2 [==============================] - 0s 53ms/step - loss: 10.2395 - val_loss: 11.0232
Epoch 12/2

In [40]:
print("error")
print(error_scores)
error=np.mean(error_scores)
print("\n")
print(error)

error
[8.729194641113281, 10.362403869628906, 11.464653015136719, 10.89039134979248, 20.240612030029297]


12.337450981140137
